In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import ast

In [38]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('../asm1/portfolio_optimization_results.csv')


In [39]:
# Get the portfolio with the highest optimized objective value
best_portfolio = df.loc[df['Optimized_Objective'].idxmax()]
best_portfolio

Combination                   ('SUN', 'BWP', 'CPU', 'VEA', 'CSL', 'GNC')
Optimized_b            [0.21685176406, 0.1, 0.2461669391, 0.236980723...
Optimized_Objective                                             9.030287
Optimized_Returns                                               0.000705
Minimum_Variance                                                0.000078
Name: 16, dtype: object

Store the data

In [40]:
# Convert the Series to a DataFrame
df_to_save = pd.DataFrame(best_portfolio).T

# Store the DataFrame into a CSV file
df_to_save.to_csv('best_portfolio.csv', index=False)

## Cleaning

In [25]:
df = pd.read_csv('best_portfolio.csv')

# Convert the 'Optimized_b' column from strings to lists of floats
df['Optimized_b'] = df['Optimized_b'].apply(lambda x: np.fromstring(x[1:-1], sep=', '))

# Multiply the expected returns by 100
df['Optimized_Returns'] *= 100
# Multiply the minimum variance by 100 squared
df['Minimum_Variance'] *= 100**2
# Calculate the standard deviation from the variance
df['Standard_Deviation'] = np.sqrt(df['Minimum_Variance'])

# round up
df['Optimized_b'] = df['Optimized_b'].apply(lambda x: np.round(x, 4))

df

,Combination,Optimized_b,Optimized_Objective,Optimized_Returns,Minimum_Variance,Standard_Deviation
0,"('SUN', 'BWP', 'CPU', 'VEA', 'CSL', 'GNC')","[0.2169, 0.1, 0.2462, 0.237, 0.1, 0.1]",9.030287,0.070545,0.781207,0.883859


In [27]:
# Obtain the optimised obj portfolio
portfolio_combination = list(
    ast.literal_eval(df['Combination'][0])
)
portfolio_weights = list(df['Optimized_b'][0])
# Compute the log return
stock_returns = pd.read_csv('holdingperiod_log_returns.csv', index_col='Date')
stock_returns.index = pd.to_datetime(stock_returns.index)
portfolio_return = np.dot(stock_returns[portfolio_combination].values, portfolio_weights)
portfolio_return = pd.DataFrame(portfolio_return, index=stock_returns.index, columns=['Portfolio Return'])

C:\Users\Windows\AppData\Local\Temp\ipykernel_17600\2605093979.py:8: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  stock_returns.index = pd.to_datetime(stock_returns.index)


In [29]:
stock_returns['Portfolio'] = portfolio_return

stock_returns.to_csv('holdingperiod_log_returns.csv')

In [24]:
stock_returns = pd.read_csv('holdingperiod_log_returns.csv')
# extract the date part from each string in the 'Date' column
stock_returns['Date'] = stock_returns['Date'].str.extract(r'(\d{4}-\d{2}-\d{2})')
# Convert the 'Date' column to datetime format
stock_returns['Date'] = pd.to_datetime(stock_returns['Date'], errors='coerce')
# Extract only the date part
stock_returns['Date'] = stock_returns['Date'].dt.date

stock_returns.set_index('Date', inplace=True)
stock_returns.to_csv('holdingperiod_log_returns.csv')